In [1]:
import pandas as pd

## CTR estimation based on clicks history

In [2]:
def clicksPerAd(clicks, group_by=['ad_id', 'adsPerDisplay'], alpha=10, beta=0.1):

    res = pd.DataFrame({'clicked': clicks.groupby(group_by)['clicked'].sum(),
                         'viewed': clicks.groupby(group_by)['clicked'].size()
                          }).reset_index()
    
    res_mean = clicks['clicked'].mean()
   
    by = group_by[1:]

    clicksPerAd_train_means = pd.DataFrame({'mean':clicks.groupby(by)['clicked'].mean()}).reset_index()
    res = res.merge(clicksPerAd_train_means, on=by)
    res['clicksPerShows'] = (beta*res_mean + alpha*res['mean'] + res['clicked']) / (beta + alpha + res['viewed'])
    res.drop(['clicked', 'viewed', 'mean'], axis=1, inplace=True)
        
    return res

In [3]:
events_clicks_train = pd.read_csv("../generated/final/events_clicks_train.csv", usecols=['display_id', 'ad_id', 'adsPerDisplay', 'clicked'])
events_clicks_train.count()

display_id       87141731
ad_id            87141731
clicked          87141731
adsPerDisplay    87141731
dtype: int64

In [4]:
clicksPerAd_train = clicksPerAd(events_clicks_train,  group_by=['ad_id', 'adsPerDisplay'], alpha=10, beta=0.1)
clicksPerAd_train.count()

ad_id             1570987
adsPerDisplay     1570987
clicksPerShows    1570987
dtype: int64

In [5]:
events_clicks_train = events_clicks_train.merge(clicksPerAd_train, how='left', on = ['ad_id', 'adsPerDisplay'])
events_clicks_train.count()

display_id        87141731
ad_id             87141731
clicked           87141731
adsPerDisplay     87141731
clicksPerShows    87141731
dtype: int64

In [6]:
events_clicks_train = events_clicks_train.drop(['clicked', 'adsPerDisplay'], axis=1)
events_clicks_train = events_clicks_train.sort_values(by=['display_id', 'clicksPerShows'], ascending=[True, False])

In [7]:
events_clicks_train.to_csv("../generated/final/events_CTR_train.csv", index=False)

In [8]:
del events_clicks_train

In [9]:
clicks_test = pd.read_csv("../generated/final/events_clicks_test.csv", usecols=['display_id', 'ad_id', 'adsPerDisplay'])
clicks_test.count()

display_id       32225162
ad_id            32225162
adsPerDisplay    32225162
dtype: int64

In [10]:
prediction = pd.merge(clicks_test, clicksPerAd_train, how='left', on = ['ad_id', 'adsPerDisplay'])
prediction.count()

display_id        32225162
ad_id             32225162
adsPerDisplay     32225162
clicksPerShows    30737615
dtype: int64

In [11]:
del clicks_test

In [12]:
clicksPerAd_train_means = pd.DataFrame({'clicksPerShows':clicksPerAd_train.groupby(['adsPerDisplay'])['clicksPerShows'].mean()}).reset_index()
del clicksPerAd_train

In [13]:
index = pd.isnull(prediction['clicksPerShows'])
prediction_nulls = prediction[index].copy()
prediction_nulls.drop(['clicksPerShows'], axis=1, inplace=True)
prediction_nulls = prediction_nulls.merge(clicksPerAd_train_means, how='left', on = ['adsPerDisplay'])
prediction = pd.concat([prediction[~index], prediction_nulls])
prediction = prediction.sort_values(by=['display_id', 'clicksPerShows'], ascending=[True, False])
prediction.count()

display_id        32225162
ad_id             32225162
adsPerDisplay     32225162
clicksPerShows    32225162
dtype: int64

In [14]:
prediction.drop('adsPerDisplay', axis=1).to_csv("../generated/final/events_CTR_test.csv", index=False)

In [15]:
del prediction
del prediction_nulls

In [16]:
pd.read_csv("../generated/final/events_CTR_train.csv", nrows=10).head()

,display_id,ad_id,clicksPerShows
0,1,279295,0.312033
1,1,144739,0.289011
2,1,42337,0.148100
3,1,139684,0.139921
4,1,296965,0.088274


In [17]:
pd.read_csv("../generated/final/events_CTR_test.csv", nrows=10).head()

,display_id,ad_id,clicksPerShows
0,16874594,170392,0.312005
1,16874594,172888,0.271547
2,16874594,162754,0.203940
3,16874594,150083,0.067184
4,16874594,66758,0.043269
